## Using the StableBaselines3 library for reinforcement learning

In this notebook we test an implementation of the proximal policy optimization (PPO)
PPO is described in detail in https://arxiv.org/abs/1707.06347. It is a variant of Trust Region Policy Optimization (TRPO) described (in this paper )[https://arxiv.org/abs/1502.05477]. The PPO algorithm works in two phases. In one phase, a large number of rollouts are performed (in parallel). The rollouts are then aggregated on the driver and a surrogate optimization objective is defined based on those rollouts. We then use SGD to find the policy that maximizes that objective with a penalty term for diverging too much from the current policy.

![ppo](https://raw.githubusercontent.com/ucbrise/risecamp/risecamp2018/ray/tutorial/rllib_exercises/ppo.png)

We begin by installing Python 3.8 in our environment, mounting Google Drive and cloning the repository with the 3D bin packing environment (only if you are using Google Colab).

In [2]:
!sudo apt-get install python3.8

#change alternatives
!sudo update-alternatives --install /usr/bin/python3  python3/usr/bin/python3.7.1
!sudo update-alternatives --install /usr/bin/python3 python3/usr/bin/python3.8.2

#check python version
!python --version

#install pip
!sudo apt-get install python-pip
!sudo apt install python3.8-distutils

#force reinstall pip (fixes issue with pip not working)
#see: https://askubuntu.com/questions/1025189/pip-is-not-working-importerror-no-module-named-pip-internal
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python3 get-pip.py --force-reinstall


Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3.8 is already the newest version (3.8.10-0ubuntu1~20.04.6).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
update-alternatives: --install needs <link> <name> <path> <priority>

Use 'update-alternatives --help' for program usage information.
update-alternatives: --install needs <link> <name> <path> <priority>

Use 'update-alternatives --help' for program usage information.
Python 3.8.10
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package python-pip is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  python3-pip

E: Package 'python-pip' has no installation candidate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, s

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
%cd /content/drive/MyDrive
!rm -rf /Github/3D-bin-packing
!git clone https://github.com/luisgarciar/3D-bin-packing.git
%cd Github/3D-bin-packing/
!git checkout dev-luis

/content/drive/MyDrive
fatal: destination path '3D-bin-packing' already exists and is not an empty directory.
/content/drive/MyDrive/Github/3D-bin-packing
fatal: reference is not a tree: dev-luis


In [10]:
!git checkout dev-luis

fatal: reference is not a tree: dev-luis


We now install the required libraries from the requirements.txt file from the repository. 

In [ ]:
!pip install -r /content/drive/MyDrive/Github/3D-bin-packing/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached nptyping-2.3.1-py3-none-any.whl (32 kB)
  Using cached numpy-1.21.5-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached Pillow-9.2.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached plotly-5.9.0-py2.py3-none-any.whl (15.2 MB)
  Using cached pytest-7.1.2-py3-none-any.whl (297 kB)
  Using cached sb3_contrib-1.6.0-py3-none-any.whl (78 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: pluggy
    Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Attempting uninstall: pytest
    Found existing installation: pytest 3.6.4
    Uninstalling pytest-3.6.4:
      Successfully uninstalled pytest-3.6.4
  Attempting uninstall: plotl

In [ ]:
!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib


Next we add the path to be able to find the environment files.

In [ ]:
import sys
import os

py_file_location = "/content/drive/MyDrive/Github/3D-bin-packing"
sys.path.append(os.path.abspath(py_file_location))

We now test the PPO algorithm with the 3D bin packing environment.

In [ ]:
import warnings

import gym
from numpy.typing import NDArray
from sb3_contrib.common.wrappers import ActionMasker
from sb3_contrib.ppo_mask import MaskablePPO
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.env_util import make_vec_env

from src.utils import boxes_generator


In [ ]:

def mask_fn(env: gym.Env) -> NDArray:
    return env.get_action_mask


def make_env(container_size, num_boxes, num_visible_boxes=1, seed=0):
    """
    Utility function for initializing bin packing env with action masking
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """

    env = gym.make(
        "PackingEnv-v0",
        container_size=container_size,
        box_sizes=boxes_generator(container_size, num_boxes, seed),
        num_visible_boxes=num_visible_boxes,
    )
    env = ActionMasker(env, mask_fn)
    return env

In [ ]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
# Environment initialization
container_size = [5, 5, 5]
num_boxes = 10
num_visible_boxes = 10
num_env = 4
env_kwargs = dict(
    container_size=container_size,
    num_boxes=num_boxes,
    num_visible_boxes=num_visible_boxes,
    seed=42,
    render_mode="rgb_array",
)
env = make_vec_env(make_env, n_envs=num_env, env_kwargs=env_kwargs)
print("finished initialization of vectorized environment")
print("beginning training")

# MaskablePPO initialization
model = MaskablePPO("MultiInputPolicy", env, gamma=0.4, verbose=1)
checkpoint_callback = CheckpointCallback(
    save_freq=50, save_path="../logs/", name_prefix="rl_model"
)
model.learn(5000, callback=checkpoint_callback)
print("done training")
model.save("ppo_mask")

Process ForkServerProcess-13:
Process ForkServerProcess-14:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/envs/3D-bin-packing/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/3D-bin-packing/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/3D-bin-packing/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/3D-bin-packing/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/3D-bin-packing/lib/python3.9/site-packages/stable_baselines3/common/vec_env/subproc_vec_env.py", line 27, in _worker
    env = env_fn_wrapper.var()
  File "/opt/anaconda3/envs/3D-bin-packing/lib/python3.9/site-packages/stable_baselines3/common/vec_env/subproc_vec_env.py", line 27, in _worker
 

KeyboardInterrupt: 

In [ ]:
# obs = env.reset()
# while True:
#     # Retrieve current action mask
#     action_masks = get_action_masks(env)
#     action, _states = model.predict(obs, action_masks=action_masks)
#     obs, rewards, dones, info = env.step(action)
#     env.render()
